In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
#como el navegador a usar es chrome, hemos descargado ya el driver de la página, guardamos la ruta en una variable
#chrome_path = 'chromedriver_linux64/chromedriver'
#driver = webdriver.Chrome(executable_path = chrome_path)

In [2]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36'}

¿Qué queremos obtener de la página?
1. Nombre del producto
2. Precio (?)
3. Detalles
4. Ingredientes

## 1 OBTENER TODAS LAS URLS DE LOS PRODUCTOS

Para ello, utilizaremos bucles que iteren sobre los elementos de la página.
* Variable global cleansers_url, es el contenedor de todas las urls.
* Los links están dentro de un tag 'a', como sólo nos interesan aquellas que tengan href, pues href=True. 
* Se ha agregado class='product-image" porque si no, el bucle cogía todas las urls (wishlist, url de la imagen etc), y solo nos interesan los productos.
* Append cada uno de los links al contenedor.

In [3]:
essence_url = []

In [4]:
for i in range (1,64):
    url = f'https://www.stylevana.com/en_EU/skincare/face-care/essence-serum.html?p={i}' 
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'lxml')
    essence_list = soup.find_all('div', class_='product-item-info')
    for item in essence_list:
        for link in item.find_all('a', class_='product-image', href=True): 
            essence_url.append(link['href'])
print(len(essence_url))

755


with open("cleansers_url.txt", "w") as output:
    output.write(str(cleansers_url))

In [7]:
toner_url=essence_url

## 2 OBTENER LA INFO DE LA PÁGINA PROPIA DEL PRODUCTO
Info que necesitamos:
* Nombre del producto
* Details
* Ingredients

* Details e Ingredients se encuentran dentro de un grupo <div class='panel-group'

In [8]:
toners = []

In [15]:
for link in toner_url[732:793]:
    r = requests.get(link, headers=headers) #para cada link en el listado, hacemos una requests
    soup = BeautifulSoup(r.content, 'lxml')
    #usamos try, except porque hay ocasiones en las que no hay info sobre los ingredientes
    try:
        name = soup.find('h1', class_='product-name-h1').text
        details = (soup.find('div', id='product-view-details').text.strip()) # details
        ingredients = soup.find_all('div', id="product-view-ingredients") #ingredients
        for item in ingredients:
            ingredients = item.find('div', class_='panel-body').text.strip()
    except:
        name = 'no info'
        details = 'no info'
        ingredients = 'no info'
        
    toners_info = {
    'name': name,
    'details': details,
    'ingredients': ingredients}
    
    toners.append(toners_info)

In [17]:
print(len(toners))

755


## 3. PASARLO A DATAFRAME

In [21]:
essence_df = pd.DataFrame(toners)
essence_df

,name,details,ingredients
0,BEAUTY OF JOSEON - Glow Serum : Propolis + Nia...,"Cruelty Free: Yes\n\n\nUsage: Day, Night\n\n\n...",[]
1,COSRX - Advanced Snail 96 Mucin Power Essence ...,"Cruelty Free: Yes\n\n\nUsage: Day, Night\n\n\n...","Snail Secretion Filtrate, Betaine, Butylene Gl..."
2,Krave - Great Barrier Relief - 45ml,"Usage: Day, Night\n\n\nProduct Type: Essence &...",[]
3,BEAUTY OF JOSEON - Ginseng Essence Water - 150...,"Cruelty Free: Yes\n\n\nUsage: Day, Night\n\n\n...","Panax ginseng root water, Butylene glycol, Gly..."
4,BEAUTY OF JOSEON - Repair Serum : Ginseng + Sn...,"Cruelty Free: Yes\n\n\nUsage: Day, Night\n\n\n...",[]
...,...,...,...
750,White Formula - Extreme Hyaluronic Acid Essenc...,"Usage: Day, Night\n\n\nProduct Type: Essence &...",[]
751,Fancl - Pore Essence - 8g,"Usage: Day, Night\n\n\nProduct Type: Essence &...","Water, Cyclopentasiloxane, Silica, Butylene Gl..."
752,Beaudiani - Organic Jojoba Oil - 100ml,"Usage: Day, Night\n\n\nProduct Type: Oil\n\n\n...",100% organic jojoba oil
753,Saturday Skin - Beauty Essence - 50ml,"Cruelty Free: Yes\n\n\nUsage: Day, Night\n\n\n...",[]


In [22]:
essence = essence_df.to_csv('data/essence.csv')